In [5]:
import pandas as pd
import numpy as np
import nltk
import re

In [6]:
5+6

11

In [7]:
import nltk
nltk.__version__

'3.8.1'

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB


In [9]:
# df = pd.read_csv("training.csv", encoding="ISO-8859-1")
df = pd.read_csv("sentiment140_with_location.csv", header=None, names=["target", "unique_id", "date", "flag", "user_id", "text","location"], encoding="ISO-8859-1")
df.sample(10)

,target,unique_id,date,flag,user_id,text,location
1569029,4,2188323559,Mon Jun 15 21:41:24 PDT 2009,NO_QUERY,JackBear,@micklpickl Netherlands card cool -- Two Lips ...,Italy
764303,0,2298792664,Tue Jun 23 12:07:05 PDT 2009,NO_QUERY,AllyCatB,"@mslisaitsnthn I'm great, just taking things 1...",Ethiopia
872380,4,1679005757,Sat May 02 08:17:55 PDT 2009,NO_QUERY,AJansa,"@ithinkminh well, you have till Oct to get there",Nigeria
287305,0,1994179399,Mon Jun 01 11:32:14 PDT 2009,NO_QUERY,Maewolf,ugh...all water...no coke...water made my stom...,Dominican Republic
1346730,4,2044381879,Fri Jun 05 09:16:31 PDT 2009,NO_QUERY,stephmassey,Tooth Fairy 4got to leave $ 4 Carson. So Palme...,Dominican Republic
79591,0,1751912823,Sat May 09 20:35:09 PDT 2009,NO_QUERY,sentience,My grandfather died last night.,Australia
1347246,4,2044500399,Fri Jun 05 09:27:01 PDT 2009,NO_QUERY,jonpeart,@jr_on_the_cloud how's the move going? Any da...,Ethiopia
1054553,4,1962019926,Fri May 29 10:23:57 PDT 2009,NO_QUERY,wxnerd,@mightyflorist Please report back on your find...,Antarctica
529495,0,2195487223,Tue Jun 16 11:11:47 PDT 2009,NO_QUERY,angelaaah,@brigglesy2k I wish. I have a guide dog meetin...,Ethiopia
115710,0,1827191869,Sun May 17 10:11:14 PDT 2009,NO_QUERY,inkinydinkynora,"@ElkaBr shall we try one last time, im gutted ...",Ethiopia


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 7 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   target     1600000 non-null  int64 
 1   unique_id  1600000 non-null  int64 
 2   date       1600000 non-null  object
 3   flag       1600000 non-null  object
 4   user_id    1600000 non-null  object
 5   text       1600000 non-null  object
 6   location   1600000 non-null  object
dtypes: int64(2), object(5)
memory usage: 85.4+ MB


In [11]:
df.shape

(1600000, 7)

In [12]:
df.drop(['unique_id','date','flag','user_id'], axis = 1, inplace = True) 

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   target    1600000 non-null  int64 
 1   text      1600000 non-null  object
 2   location  1600000 non-null  object
dtypes: int64(1), object(2)
memory usage: 36.6+ MB


In [14]:
df.sample(10)

,target,text,location
1102149,4,@LittleLiverbird ooh! I like that car. I love ...,Canada
937042,4,@kimluvsdonnie yay would love to meet all the...,Nigeria
370476,0,I want to go to pinehurst!!,Dominican Republic
415789,0,@shaybreezy i know me too i just keep thinkin...,Peru
849900,4,@SasaLoves Happy Brithday Eve!!!!,UK
14548,0,"Wet hair + No Hair Dryer = ... Dry fucker, DR...",Tanzania
451243,0,@tommcfly,Indonesia
1532514,4,@fromduval Thank you,Tanzania
143497,0,Phewww stayed up all night to finish my floorp...,Italy
759991,0,mad that holly got off of celebrity survivor,Brazil


In [15]:
df.rename(columns={'target': 'sentiment', 'text': 'tweet'}, inplace = True)

In [16]:
df.sample(10)

,sentiment,tweet,location
1071451,4,@MrInfamy you've done smething I really really...,Jamaica
1297119,4,Working &amp; a meeting at the remote office (...,Brazil
10325,0,Why do only two of my friends have twitter! on...,Mexico
1006514,4,needs more sleep please,Spain
383256,0,is at home...with no one to play with,Jamaica
1222024,4,@Jonasbrothers Heyahh Jonas Brothers! Goodluck...,Australia
678040,0,@tracy_nyc I'm gonna guess &amp; say the final...,Australia
1446821,4,Welcome to all my new followers HUGS to you all,New Zealand
42727,0,Just got outta work. Gf wants to go out. But ...,Tanzania
1535396,4,@ReikiAwakening sea salt i hope,Dominican Republic


In [17]:
df = df.dropna()
df.sample(10)

,sentiment,tweet,location
523359,0,ha i hate mean people too,Egypt
625689,0,"@titania1108 No, around the corner from us. Wi...",Brazil
556504,0,@bethycotter i will miss your twitter updates ...,France
1414630,4,@sandipantz lucky lucky woman! so excited for ...,Japan
471581,0,@zatyiqlima yeahh i knw. i missed baby so much,Chile
291084,0,getting ready for school ! Yo I'm so nervous a...,Indonesia
149248,0,Listening to the last half hour of my favorite...,South Korea
509026,0,Headache,Colombia
1135928,4,@aldriniscool Awesome. I have the camera they ...,Ethiopia
1008122,4,@mhaemee : I'm from Vietnam,Chile


In [18]:
print(df.info())
print("The Shape Of The Dataset Is :",df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1600000 non-null  int64 
 1   tweet      1600000 non-null  object
 2   location   1600000 non-null  object
dtypes: int64(1), object(2)
memory usage: 36.6+ MB
None
The Shape Of The Dataset Is : (1600000, 3)


In [19]:
df['sentiment'] = df['sentiment'].replace({0: 'negative', 4: 'positive'})

In [20]:
df.sample(5)

,sentiment,tweet,location
1249945,positive,The Sims 3 tomorrow!,Italy
569404,negative,Ok Im off to Unem office,Mexico
1435177,positive,"@swallowedeasy weelllcome, i love the kstewart...",South Korea
822876,positive,Home for Easter,Egypt
1283223,positive,"Mornin all.... ooh, sun looks lovely outside!...",Japan


In [21]:
# Remove URLs, special characters, punctuation, and numbers from tweet column
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub('[^a-zA-Z\s]', '', x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub('[0-9]+', '', x))


In [22]:
print(df.sample(10))
print('\033[1m' + 'The information of the dataset is :' + '\033[0m')
print(df.info())
print('\033[1m' + 'The Shape Of The Dataset Is :' + '\033[0m',df.shape)

        sentiment                                              tweet  location
1038753  positive  Every dog has hisher day but that doesnt mean ...     Kenya
905283   positive  agriggs is what it takes I am soooo dragging h...     Spain
548014   negative                                bloody car battery      Chile
1062225  positive           masonpants She says Im a shit  and hello  Tanzania
1286694  positive  Retwittable webdesign related Links for you   ...     Japan
756949   negative  laying in the dark listening to saturdaysyouth...       USA
1324319  positive  LeShack Youre over the age of  and you still d...      Peru
1517964  positive  Workin on gettng the good ol Facebook account ...    France
908841   positive    At the apartment nothin to do a little hungry       Japan
1347538  positive                   johnnykile that was really deep    Ecuador
The information of the dataset is :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (t

In [23]:
df['tweet'] = df['tweet'].str.lower()

In [24]:
df.sample(10)

,sentiment,tweet,location
150018,negative,i wish adam won,Jamaica
1553068,positive,i am musical today,USA
147910,negative,can anybody make friendeed work in nambu on th...,Canada
1111375,positive,todays the day graduation,Ethiopia
1307159,positive,whats on for today volunteering at saf hub the...,India
514348,negative,calevans try safari with blending colors cras...,New Zealand
1249026,positive,repressd bring your scentsy,Argentina
1154272,positive,oscartg more like a bloody good idea liking y...,Australia
59649,negative,tired and feeling rough,Tanzania
1031712,positive,peachez lol,Egypt


In [25]:
# stop_words = set(stopwords.words('english'))
# df['tweet'] = df['tweet'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))